In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
#from sklearn.cross_validation import train_test_split


import sys
sys.path.append("..")

In [19]:
customers = pd.read_csv('recommend_1.csv') 
transactions = pd.read_csv('trx_data.csv')

In [7]:
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(),
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})


 pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products')

transactions['products'][2].split("|")

In [22]:
transactions1 = transactions.reset_index()

In [24]:
transactions1.columns = ['trns', 'customerId', 'products']

In [31]:
new_df =pd.DataFrame(transactions1.products.str.split('|').tolist(), index=transactions1.trns).stack()

In [35]:
df_v3 = pd.DataFrame(new_df).reset_index()

,trns,level_1,0
0,0,0,20
1,1,0,2
2,1,1,2
3,1,2,23
4,1,3,68
...,...,...,...
211473,62479,8,35
211474,62479,9,8
211475,62480,0,24
211476,62481,0,167


In [36]:
df_v3.columns = ['trns', 'level_1', 'productId']

In [39]:
new_df_v2 = df_v3[['trns', 'productId']].merge(transactions1[['trns', 'customerId']], how='left', on = 'trns')

,trns,productId,customerId
0,0,20,0
1,1,2,1
2,1,2,1
3,1,23,1
4,1,68,1
...,...,...,...
211473,62479,35,10918
211474,62479,8,10918
211475,62480,24,6119
211476,62481,167,8171


In [41]:
data = new_df_v2.groupby(['customerId', 'productId']) \
    .agg({'trns': 'count'}).reset_index()

In [62]:
data.columns = ['customerId', 'productId', 'purchase_count']

In [66]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [67]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [68]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [69]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [70]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [71]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [73]:
import sklearn.model_selection

In [74]:
from math import sqrt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt



In [75]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [76]:
# constant variables to define field names include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [77]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model


In [78]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23298 users and 300 items.

Data prepared in: 0.163524s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 96899.2

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    248    |       3.1875       |  1   |
|    1553    |    132    | 3.1818181818181817 |  2   |
|    1553    |     34    | 3.039525691699605  |  3   |
|    1553    |     37    | 2.9840637450199203 |  4   |
|    1553    |     0     | 2.9752851711026618 |  5   |
|    1553    |     3     | 2.795309168443497  |  6   |
|    1553    |     27    | 2.7829457364341086 |  7   |
|    1553    |    110    | 2.706896551724138  |  8   |
|    1553    |     82    | 2.6781115879828326 |  9   |
|    1553    |     32    | 2.6411483253588517 |  10  |
|   20400    |    248    |       3.1875       |  1   |
|   20400    |    132    | 3.1818181818181817 |  2   |
|   20400    |     34    | 3.039525691699605  |  3   |
|   20400    |     37    | 2.9840637450199203 |  4   |
|   20400    |     0     | 2.9752851711026618 |  5   |
|   20400 

In [79]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23339 users and 300 items.

Data prepared in: 0.139209s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 93353.2

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    159    |  1.0  |  1   |
|    1553    |    215    |  1.0  |  2   |
|    1553    |     4     |  1.0  |  3   |
|    1553    |    257    |  1.0  |  4   |
|    1553    |    216    |  1.0  |  5   |
|    1553    |    244    |  1.0  |  6   |
|    1553    |     33    |  1.0  |  7   |
|    1553    |    152    |  1.0  |  8   |
|    1553    |    154    |  1.0  |  9   |
|    1553    |     46    |  1.0  |  10  |
|   20400    |    159    |  1.0  |  1   |
|   20400    |    215    |  1.0  |  2   |
|   20400    |     4     |  1.0  |  3   |
|   20400    |    257    |  1.0  |  4   |
|   20400    |    216    |  1.0  |  5   |
|   20400    |    244    |  1.0  |  6   |
|   20400    |     33    |  1.0  |  7   |
|   20400    |    152    |  1.0  |  8   |
|   20400    |    154    |  1.0  |  9   |
|   20400    |     46    |  1.0  |  10  |
|   19750    |    159    |  1.0  |

In [80]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23288 users and 300 items.

Data prepared in: 0.152586s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 86348.3

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7653061224489796 |  1   |
|    1553    |    247    |  0.3333333333333334 |  2   |
|    1553    |    230    | 0.33233082706766853 |  3   |
|    1553    |    248    |  0.2616279069767442 |  4   |
|    1553    |    125    |  0.2589147286821702 |  5   |
|    1553    |    294    |  0.2519685039370075 |  6   |
|    1553    |    276    |  0.2354014598540146 |  7   |
|    1553    |    204    | 0.22954545454545439 |  8   |
|    1553    |     72    |  0.2268041237113402 |  9   |
|    1553    |    132    | 0.22638146167557932 |  10  |
|   20400    |    226    |  0.7653061224489796 |  1   |
|   20400    |    247    |  0.3333333333333334 |  2   |
|   20400    |    230    | 0.33233082706766853 |  3   |
|   20400    |    248    |  0.2616279069767442 |  4   |
|   20400    |    125    |  0.2589147286821702 |

In [ ]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending = False).head(20)

# Cosine similarity

### Using purchase count

In [83]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23298 users and 300 items.

Data prepared in: 0.184129s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.05705s

recommendations finished on 1000/1000 queries. users per second: 25176.9

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11221296787261963  |  1   |
|    1553    |     17    | 0.07446370124816895  |  2   |
|    1553    |     1     | 0.06931562423706054  |  3   |
|    1553    |     35    | 0.06675516366958618  |  4   |
|    1553    |    162    | 0.06379594802856445  |  5   |
|    1553    |     5     | 0.06333938837051392  |  6   |
|    1553    |     33    | 0.051913213729858396 |  7   |
|    1553    |     8     | 0.045176589488983156 |  8   |
|    1553    |     9     | 0.04410766363143921  |  9   |
|    1553    |    167    | 0.04236369132995606  |  10  |
|   20400    |    280    | 0.08197379112243652  |  1   |
|   20400    |    265    | 0.04445701837539673  |  2   |
|   20400    |     31    | 0.04356563091278076  |  3   |
|   20400    |    122    | 0.04136162996292114  |  4   |
|   20400    |     1     | 0.03

### Using purchase dummy

In [84]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23339 users and 300 items.

Data prepared in: 0.216925s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.097236s

recommendations finished on 1000/1000 queries. users per second: 24277.7

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.09552706480026245  |  1   |
|    1553    |     1     | 0.08511338233947754  |  2   |
|    1553    |     5     | 0.07500290870666504  |  3   |
|    1553    |     35    | 0.07359938621520996  |  4   |
|    1553    |     21    | 0.06205728054046631  |  5   |
|    1553    |     17    | 0.06094403266906738  |  6   |
|    1553    |     8     | 0.053420627117156984 |  7   |
|    1553    |     61    | 0.05313411951065063  |  8   |
|    1553    |     47    | 0.051540148258209226 |  9   |
|    1553    |     33    | 0.04828900098800659  |  10  |
|   20400    |    122    |  0.0520363450050354  |  1   |
|   20400    |     26    | 0.04943197965621948  |  2   |
|   20400    |     13    | 0.04233652353286743  |  3   |
|   20400    |     1     |  0.0401497483253479  |  4   |
|   20400    |    265    | 0.03

### Using scaled purchase count

In [85]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23288 users and 300 items.

Data prepared in: 0.163586s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.087742s

recommendations finished on 1000/1000 queries. users per second: 40196.2

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |     25    |          0.0          |  1   |
|    1553    |    113    |          0.0          |  2   |
|    1553    |     76    |          0.0          |  3   |
|    1553    |    138    |          0.0          |  4   |
|    1553    |     2     |          0.0          |  5   |
|    1553    |     35    |          0.0          |  6   |
|    1553    |    236    |          0.0          |  7   |
|    1553    |     1     |          0.0          |  8   |
|    1553    |     13    |          0.0          |  9   |
|    1553    |     17    |          0.0          |  10  |
|   20400    |     25    |          0.0          |  1   |
|   20400    |    113    |          0.0          |  2   |
|   20400    |     76    |          0.0          |  3   |
|   20400    |    138    |          0.0          |  4   |
|   20400    |

# Pearson similarity

### Using purchase count

In [86]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23298 users and 300 items.

Data prepared in: 0.175223s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.114689s

recommendations finished on 1000/1000 queries. users per second: 28210.3

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    248    | 3.184131479263305  |  1   |
|    1553    |    132    | 3.181818181818181  |  2   |
|    1553    |     34    | 3.0229726593013817 |  3   |
|    1553    |     37    | 2.982912505147942  |  4   |
|    1553    |     0     | 2.971873842310086  |  5   |
|    1553    |     3     | 2.7953091684434956 |  6   |
|    1553    |     27    | 2.782945736434109  |  7   |
|    1553    |    110    | 2.704622288819017  |  8   |
|    1553    |     82    | 2.676156245690046  |  9   |
|    1553    |     32    | 2.6411483253588517 |  10  |
|   20400    |    248    | 3.1850231289863578 |  1   |
|   20400    |    132    | 3.172295743768865  |  2   |
|   20400    |     34    | 3.0395256916996054 |  3   |
|   20400    |     37    | 2.9840637450199208 |  4   |
|   20400    |     0     | 2.9734296189061555 |  5   |
|   20400 

### Using purchase dummy

In [87]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23339 users and 300 items.

Data prepared in: 0.175244s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.042962s

recommendations finished on 1000/1000 queries. users per second: 47415.8

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    159    |  0.0  |  1   |
|    1553    |    215    |  0.0  |  2   |
|    1553    |     4     |  0.0  |  3   |
|    1553    |    257    |  0.0  |  4   |
|    1553    |    216    |  0.0  |  5   |
|    1553    |    244    |  0.0  |  6   |
|    1553    |     33    |  0.0  |  7   |
|    1553    |    152    |  0.0  |  8   |
|    1553    |    154    |  0.0  |  9   |
|    1553    |     46    |  0.0  |  10  |
|   20400    |    159    |  0.0  |  1   |
|   20400    |    215    |  0.0  |  2   |
|   20400    |     4     |  0.0  |  3   |
|   20400    |    257    |  0.0  |  4   |
|   20400    |    216    |  0.0  |  5   |
|   20400    |    244    |  0.0  |  6   |
|   20400    |     33    |  0.0  |  7   |
|   20400    |    152    |  0.0  |  8   |
|   20400    |    154    |  0.0  |  9   |
|   20400    |     46    |  0.0  |  10  |
|   19750    |    159    |  0.0  |

### Using scaled purchase count

In [88]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23288 users and 300 items.

Data prepared in: 0.140616s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.045255s

recommendations finished on 1000/1000 queries. users per second: 84559.4

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7651816202669729 |  1   |
|    1553    |    247    |  0.3333333333333333 |  2   |
|    1553    |    230    | 0.33212310488062713 |  3   |
|    1553    |    248    |  0.2616279069767441 |  4   |
|    1553    |    125    | 0.25872835687888684 |  5   |
|    1553    |    294    |  0.2519685039370078 |  6   |
|    1553    |    276    | 0.23532564274586032 |  7   |
|    1553    |    204    |  0.2294592304663225 |  8   |
|    1553    |     72    | 0.22651251394724112 |  9   |
|    1553    |    132    | 0.22638146167557927 |  10  |
|   20400    |    226    |  0.7653061224489798 |  1   |
|   20400    |    247    |  0.3333333333333333 |  2   |
|   20400    |    230    | 0.33233082706766937 |  3   |
|   20400    |    248    |  0.2616279069767441 |  4   |
|   20400    |    125    | 0.25863488487495007 |

# Model Evaluation

In [89]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

NameError: name 'popularity_model' is not defined

In [95]:
tc.SFrame(data_norm)

customerId,productId,scaled_purchase_freq
9,0,0.13333333333333333
25,0,0.13333333333333333
33,0,0.13333333333333333
36,0,0.13333333333333333
44,0,0.13333333333333333
56,0,0.13333333333333333
60,0,0.0
72,0,0.3333333333333333
76,0,0.13333333333333333
91,0,0.0


In [96]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.212315s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.073978s

recommendations finished on 1000/1000 queries. users per second: 30828

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.12324784994125366  |  1   |
|    1553    |     35    | 0.10447167158126831  |  2   |
|    1553    |     5     |  0.0906752586364746  |  3   |
|    1553    |     1     | 0.08670114278793335  |  4   |
|    1553    |     17    | 0.07659814357757569  |  5   |
|    1553    |     8     | 0.06811234951019288  |  6   |
|    1553    |     33    |  0.0668614387512207  |  7   |
|    1553    |     21    | 0.06112504005432129  |  8   |
|    1553    |     47    | 0.06058878898620605  |  9   |
|    1553    |     61    | 0.060317397117614746 |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |     1     | 0.05210459232330322  |  3   |
|   20400    |     15    | 0.047683894634246826 |  4   |
|   20400    |     27    | 0.04

In [97]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,2,0.123248,1
1,1553,35,0.104472,2
2,1553,5,0.090675,3
3,1553,1,0.086701,4
4,1553,17,0.076598,5


In [100]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [101]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 46992.5

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|248|125|294|276|204|72|132
11,226|247|230|248|125|294|276|204|132|165
12,226|247|230|248|125|294|276|204|72|132
16,226|247|230|248|125|294|276|204|72|132
21,226|247|230|248|125|294|276|204|72|132


In [102]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [104]:
customer_recomendation(4)

recommendedProducts    226|247|230|248|125|294|276|204|72|132
Name: 4, dtype: object

In [134]:
df_products = pd.DataFrame(df_v3['productId'].unique())
df_products.columns = ['productId']
df_products['productId'] = df_products['productId'].apply(lambda x : int(x))

In [141]:
df_products_sql = df_products['productId'].sort_values().reset_index().drop('index', axis=1)

In [142]:
df_products_sql.to_csv('products_sql.csv')